In [3]:
from py2neo import Graph, authenticate, Relationship
authenticate("localhost:7474", "neo4j", "admin")
graph = Graph()

Les personnages qui apparaissent le plus souvent


In [4]:
graph.run('''
MATCH (c: character) -[r:APPEARS_IN]-> (:comics)
WITH count(r)  as res, c
WHERE  res > 100
RETURN c.name 
''').data()

[{u'c.name': u'SPIDER-MAN/PETER PAR'},
 {u'c.name': u'URICH, BEN'},
 {u'c.name': u'INVISIBLE WOMAN/SUE,'},
 {u'c.name': u'GALACTUS/GALAN'},
 {u'c.name': u'GAMBIT/REMY LEBEAU,'},
 {u'c.name': u'CAPTAIN MARVEL/CAPTA'},
 {u'c.name': u"BLACK PANTHER/T'CHAL"},
 {u'c.name': u'MEGGAN'},
 {u'c.name': u'MIRAGE II/DANIELLE M'},
 {u'c.name': u'KARNILLA [ASGARDIAN]'},
 {u'c.name': u'HAWK'},
 {u'c.name': u'SHANG-CHI'},
 {u'c.name': u'MAGIK/ILLYANA RASPUT'},
 {u'c.name': u'JAMESON, COL. JOHN,'},
 {u'c.name': u'DUGAN, TIMOTHY ALOYI'},
 {u'c.name': u'JUSTICE II/VANCE AST'},
 {u'c.name': u'NELSON, FRANKLIN FOG'},
 {u'c.name': u'LOKI [ASGARDIAN]'},
 {u'c.name': u'TRITON'},
 {u'c.name': u'KA-ZAR/KEVIN PLUNDER'},
 {u'c.name': u'SASQUATCH/WALTER LAN'},
 {u'c.name': u'BINARY/CAROL DANVERS'},
 {u'c.name': u'BISHOP /'},
 {u'c.name': u'FURY, COL. NICHOLAS,'},
 {u'c.name': u'CLEA'},
 {u'c.name': u'HUMAN TORCH/JOHNNY S'},
 {u'c.name': u'DAGGER/TANDY BOWEN'},
 {u'c.name': u'HULK/DR. ROBERT BRUC'},
 {u'c.name': u'

Relation nommée (peut être réutilisée): APPEARS_IN 
On compte le nombre de comics dans lequels apparait chaque perso
Retour par ordre decroissant
dans la limite de 10 occurences

In [5]:
graph.run('''
MATCH (c: character) -[r:APPEARS_IN]-> (:comics)
WITH count(r)  as res, c
ORDER BY res DESC
RETURN c.name as superheros, res as nbOccurrences  LIMIT 10
''').data()

[{u'nbOccurrences': 1625, u'superheros': u'SPIDER-MAN/PETER PAR'},
 {u'nbOccurrences': 1367, u'superheros': u'CAPTAIN AMERICA'},
 {u'nbOccurrences': 1168, u'superheros': u'IRON MAN/TONY STARK,'},
 {u'nbOccurrences': 990, u'superheros': u'THING/BENJAMIN J. GR'},
 {u'nbOccurrences': 965, u'superheros': u'THOR/DR. DONALD BLAK'},
 {u'nbOccurrences': 908, u'superheros': u'HUMAN TORCH/JOHNNY S'},
 {u'nbOccurrences': 875, u'superheros': u'MR. FANTASTIC/REED R'},
 {u'nbOccurrences': 841, u'superheros': u'HULK/DR. ROBERT BRUC'},
 {u'nbOccurrences': 820, u'superheros': u'WOLVERINE/LOGAN,'},
 {u'nbOccurrences': 782, u'superheros': u'INVISIBLE WOMAN/SUE,'}]

Le nombre moyen de personnages par comics

In [6]:
maList = []
# Parcours des comics
for index in range (6485,6490):
    # On teste tous les perso pour leur relation avec le comics designé par l'index
    maList.append(
    graph.run('''
    MATCH (:character) -[r: APPEARS_IN]-> (c: comics)
    WHERE c.number = {I}
    RETURN c
    ''', {"I":str(index)}).data())
maList

[[],
 [],
 [{u'c': (de8af7e:comics {name:"AA2 35",number:"6487"})},
  {u'c': (de8af7e:comics {name:"AA2 35",number:"6487"})},
  {u'c': (de8af7e:comics {name:"AA2 35",number:"6487"})},
  {u'c': (de8af7e:comics {name:"AA2 35",number:"6487"})},
  {u'c': (de8af7e:comics {name:"AA2 35",number:"6487"})},
  {u'c': (de8af7e:comics {name:"AA2 35",number:"6487"})}],
 [{u'c': (ed9caaa:comics {name:"M/PRM 35",number:"6488"})}],
 [{u'c': (e3be7ea:comics {name:"M/PRM 36",number:"6489"})}]]

In [8]:
maList = []
for index in range (6485,6490):
    maList.append(
        # Meme idee, mais au lieu de renvoyer une ligne par correspondance, 
        # on compte le nombre de relations equivalentes (un perso quelconque apparait dans ce comics particulier)
    graph.run('''
    MATCH (:character) -[r: APPEARS_IN]-> (c: comics {number :{I}} )
    WITH count(r)  as res
    RETURN  res
    ''', {"I":str(index)}).data())
maList


[[{u'res': 0}], [{u'res': 0}], [{u'res': 6}], [{u'res': 1}], [{u'res': 1}]]

In [28]:
maList = []
for index in range (6400,7000):
    maList.append(
         # On peut ecrire la relation dans l'autre sens,
        # en comptant le nombre de fois que le comics est impliqué dans une relation APPEARS_IN
        # les characters n'ont pas besoin d'être explicités
    graph.run('''
    MATCH (c: comics {number :{I}}) <-[r: APPEARS_IN]- ()  
    WITH count(r)  as res, c
    RETURN  res, c.name LIMIT 10
    ''', {"I":str(index)}).data())
    
# on obtient une liste de listes de dictionnaire
#  Utilisation de la comprehension de liste
# new_list = [function(item) for item in list if condition(item)]    
newList = [item for sublist in maList for  item in sublist]



In [13]:
# puis recuperer chaque item dans une liste
list3 = [item[u'res'] for item in newList]
list3

[0, 0, 6, 1, 1]

Rassembler le tout dans une fonction

In [22]:
np.asarray(list3).mean()

1.6000000000000001

In [31]:
sorted(newList, key=lambda dic: dic[u'res'], reverse = True)

[{u'c.name': u'COC 1', u'res': 111},
 {u'c.name': u'H2 278', u'res': 53},
 {u'c.name': u"MX '01", u'res': 49},
 {u'c.name': u'ROM 65', u'res': 42},
 {u'c.name': u'SWII 7', u'res': 41},
 {u'c.name': u'M/SHSW 12', u'res': 38},
 {u'c.name': u'M/SHSW 11', u'res': 36},
 {u'c.name': u'M/SHSW 7', u'res': 36},
 {u'c.name': u'AR 2', u'res': 35},
 {u'c.name': u'M/SHSW 1', u'res': 35},
 {u'c.name': u'M/SHSW 3', u'res': 35},
 {u'c.name': u'M/SHSW 4', u'res': 35},
 {u'c.name': u'M/SHSW 6', u'res': 35},
 {u'c.name': u'M/SHSW 8', u'res': 35},
 {u'c.name': u'Q 25', u'res': 35},
 {u'c.name': u'M/SHSW 2', u'res': 34},
 {u'c.name': u'M/SHSW 5', u'res': 33},
 {u'c.name': u'T 291', u'res': 31},
 {u'c.name': u'T 300', u'res': 31},
 {u'c.name': u'M/SHSW 10', u'res': 30},
 {u'c.name': u'UX 377', u'res': 30},
 {u'c.name': u'UX@ 10', u'res': 30},
 {u'c.name': u'NW 9', u'res': 29},
 {u'c.name': u"IM '99", u'res': 28},
 {u'c.name': u'M/SHSW 9', u'res': 28},
 {u'c.name': u'UX 223', u'res': 28},
 {u'c.name': u'FF@ 

les six degrés de Kevin Bacon pour tous les personnages par rapport à Spider-Man

In [43]:
maList=[]


for index in range(5500,6000):
    maList.append(
    graph.run('''
    MATCH p =shortestPath((n1: character{name: "SPIDER-MAN/PETER PAR"})-[r *..6]-(n2: character{number: {I}}))
    WITH length(p) as sp, n2.name as name
    RETURN sp, name ''',{"I":str(index)}).data())
maList
    

[[{u'name': u'STRYFE', u'sp': 4}],
 [{u'name': u'STRYKER, WILLIS', u'sp': 4}],
 [{u'name': u'STUART, BRIG. ALYSAN', u'sp': 4}],
 [{u'name': u'STUART, DR.', u'sp': 4}],
 [{u'name': u'STUART, DR. ALISTAIR', u'sp': 2}],
 [{u'name': u'STUNNER/ANGELINA BRA', u'sp': 2}],
 [{u'name': u'STUNT-MASTER/GEORGE,', u'sp': 2}],
 [{u'name': u'STURDY, PROF. KENNET', u'sp': 4}],
 [{u'name': u'STYGE, REV.', u'sp': 2}],
 [{u'name': u'STYGIAN STARBENDER', u'sp': 4}],
 [{u'name': u'STYGLUT/', u'sp': 4}],
 [{u'name': u'STYRAK', u'sp': 4}],
 [{u'name': u'STYX/JACOB EICHORN', u'sp': 2}],
 [{u'name': u'STYX II', u'sp': 4}],
 [{u'name': u'SU, DR. KO BIN', u'sp': 4}],
 [{u'name': u'SUAREZ, RAPHAEL', u'sp': 4}],
 [{u'name': u'SUB-MARINER/NAMOR MA', u'sp': 2}],
 [{u'name': u'SUB-MARINER | MUTANT', u'sp': 4}],
 [{u'name': u'SUEE, FAH LO', u'sp': 4}],
 [{u'name': u'SUGAR MAN', u'sp': 4}],
 [{u'name': u'SUICIDE/CHRIS DANIEL', u'sp': 4}],
 [{u'name': u'SUKHAROV, DIMITRI', u'sp': 4}],
 [{u'name': u'SUKHAROV, VIKTORIA', 

In [46]:
def relations(nomduPerso):
    maList=[]
    for index in range(5500,6000):
        maList.append(
        graph.run('''
        MATCH p =shortestPath((n1: character{name: {N}})-[r *..6]-(n2: character{number: {I}}))
        WITH length(p) as sp, n2.name as name
        RETURN sp, name ''',{"I":str(index), "N":nomduPerso}).data())
    return maList


In [47]:
relations("SPIDER-MAN/PETER PAR")

[[{u'name': u'STRYFE', u'sp': 4}],
 [{u'name': u'STRYKER, WILLIS', u'sp': 4}],
 [{u'name': u'STUART, BRIG. ALYSAN', u'sp': 4}],
 [{u'name': u'STUART, DR.', u'sp': 4}],
 [{u'name': u'STUART, DR. ALISTAIR', u'sp': 2}],
 [{u'name': u'STUNNER/ANGELINA BRA', u'sp': 2}],
 [{u'name': u'STUNT-MASTER/GEORGE,', u'sp': 2}],
 [{u'name': u'STURDY, PROF. KENNET', u'sp': 4}],
 [{u'name': u'STYGE, REV.', u'sp': 2}],
 [{u'name': u'STYGIAN STARBENDER', u'sp': 4}],
 [{u'name': u'STYGLUT/', u'sp': 4}],
 [{u'name': u'STYRAK', u'sp': 4}],
 [{u'name': u'STYX/JACOB EICHORN', u'sp': 2}],
 [{u'name': u'STYX II', u'sp': 4}],
 [{u'name': u'SU, DR. KO BIN', u'sp': 4}],
 [{u'name': u'SUAREZ, RAPHAEL', u'sp': 4}],
 [{u'name': u'SUB-MARINER/NAMOR MA', u'sp': 2}],
 [{u'name': u'SUB-MARINER | MUTANT', u'sp': 4}],
 [{u'name': u'SUEE, FAH LO', u'sp': 4}],
 [{u'name': u'SUGAR MAN', u'sp': 4}],
 [{u'name': u'SUICIDE/CHRIS DANIEL', u'sp': 4}],
 [{u'name': u'SUKHAROV, DIMITRI', u'sp': 4}],
 [{u'name': u'SUKHAROV, VIKTORIA', 

In [49]:
graph.run('''
MATCH (c: character) 
WITH COUNT(c) as nbChar
RETURN nbChar 
''').data()[0][u'nbChar']

6486

In [1]:
def relations(nomduPerso):
    maList=[]
    
    nbPersos = graph.run('''
    MATCH (c: character) 
    WITH COUNT(c) as nbChar
    RETURN nbChar 
    ''').data()[0][u'nbChar']
    
    for index in range(nbPersos):
        maList.append(
        graph.run('''
        MATCH p =shortestPath((n1: character{name: {N}})-[r *..6]-(n2: character{number: {I}}))
        WITH length(p) as sp, n2.name as name
        RETURN sp, name ''',{"I":str(index+1), "N":nomduPerso}).data())
        
    newList = [item for sublist in maList for  item in sublist]
    return sorted(newList, key=lambda dic: dic[u'sp'])



In [2]:
relations("SPIDER-MAN/PETER PAR")

NameError: global name 'graph' is not defined